In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/자동차등록대수현황_시도별_20231122095710.csv', encoding='ansi')

In [3]:
df.head(3)

,시도명(1),시군구(1),2011.01,2011.02,2011.03,2011.04,2011.05,2011.06,2011.07,2011.08,...,2023.01,2023.02,2023.03,2023.04,2023.05,2023.06,2023.07,2023.08,2023.09,2023.10
0,시도명(1),시군구(1),총계,총계,총계,총계,총계,총계,총계,총계,...,총계,총계,총계,총계,총계,총계,총계,총계,총계,총계
1,시도명(1),시군구(1),계,계,계,계,계,계,계,계,...,계,계,계,계,계,계,계,계,계,계
2,서울,계,2991173,2992406,2995826,2999731,2999402,2999651,3000038,2996185,...,3194351,3191681,3191538,3191516,3190288,3192434,3192414,3190446,3192359,3191832


In [4]:
long_df = df.melt(id_vars=['시도명(1)', '시군구(1)'], 
                  var_name='STD_YYYYMM', 
                  value_name='차량수')

In [12]:
long_df.head(3)

,SIDO,STD_YYYYMM,차량수
0,NaN,2011.01,총계
1,NaN,2011.01,계
2,서울특별시,2011.01,2991173


In [6]:
long_df.rename(columns={'시도명(1)': 'SIDO'}, inplace=True)

In [9]:

# 변경할 문자열 매핑
name_mapping = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '경기': '경기도',
    '세종': '세종특별자치시',
    '강원': '강원도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주특별자치도'
}

# 'SIDO' 컬럼의 값을 매핑에 따라 변경
long_df['SIDO'] = long_df['SIDO'].map(name_mapping)

In [11]:
long_df.drop(columns=['시군구(1)'],inplace=True)
# 시군구(1) 컬럼 삭제

In [14]:
# 'SIDO' 컬럼에서 NaN 값을 가진 행을 제거
long_df = long_df.dropna(subset=['SIDO'])

In [16]:
# 'STD_YYYYMM' 열의 숫자가 아닌 문자를 제거
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].str.replace('[^\d]', '', regex=True)

In [17]:
# 결과를 정수 타입으로 변환
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].astype(int)

In [23]:
# '차량수' 컬럼을 수치형으로 변환
long_df['차량수'] = pd.to_numeric(long_df['차량수'], errors='coerce')

In [18]:
long_df['STD_YYYYMM'].dtype

dtype('int32')

In [24]:
long_df['차량수'].dtype

dtype('float64')

In [25]:
long_df.head()

,SIDO,STD_YYYYMM,차량수
2,서울특별시,201101,2991173.0
3,부산광역시,201101,1153513.0
4,대구광역시,201101,953292.0
5,인천광역시,201101,931279.0
6,광주광역시,201101,521402.0


In [26]:
# 누락된 값이 있는 행 찾기
missing_values_df = long_df[long_df['차량수'].isna()]

# 'STD_YYYYMM' 별 '차량수' 평균값 계산
mean_values = long_df.groupby('STD_YYYYMM')['차량수'].transform('mean')

# 누락된 '차량수' 값에 평균값 할당
long_df['차량수'] = long_df['차량수'].fillna(mean_values)


In [27]:
long_df.to_csv('data/test/차량.csv', index=False)